# Master Discogs Database

In [7]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collections import Counter
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from time import sleep


################################################################################
## Music Stuff
################################################################################

### MultiArtist
from multiArtist import multiartist

### My Music DB
from myMusicDBMap import myMusicDBMap

### Master DB code
from masterdb import masterdb
from mainDB import mainDB


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-09-06 19:15:55.035792


In [8]:
mdb = myMusicDBMap(debug=True)

Creating myMusicDBMap()
   Loading my music db map: /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Found 4388 artist entries
Counter({'Discogs': 4251, 'AllMusic': 4156, 'MusicBrainz': 3952, 'LastFM': 3280, 'CDandLP': 758, 'DatPiff': 549, 'RockCorner': 507, 'AceBootlegs': 179, 'RateYourMusic': 68, 'MusicStack': 5, 'MetalStorm': 0})
Counter({'Discogs': 4251, 'AllMusic': 4156, 'MusicBrainz': 3952, 'LastFM': 3280, 'CDandLP': 758, 'DatPiff': 549, 'RockCorner': 507, 'AceBootlegs': 179, 'RateYourMusic': 68, 'MusicStack': 5, 'MetalStorm': 0})


In [14]:
maindb = mainDB(mdb=mdb, create=True, debug=True)
print("\n\n{0}\n".format("Full DB"))
#maindb.setDBFull("AllMusic")
maindb.setDBFull("MusicBrainz")
maindb.setDBFull("LastFM")
maindb.setDBFull("Discogs")
print("\n\n{0}\n".format("Known DB"))
#maindb.setDBKnown() ## Do this to recreate everything
#maindb.setDBKnown()
#artistDBs = maindb.getKnownArtistDBs()

Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rockcorner   =========================
=========================   datpiff   =========================
=========================   acebootlegs   =========================
=========================   cdandlp   =========================
=========================   rateyourmusic   =========================
=========================   musicstack   =========================
  Creating Database Records for Discogs
  Creating Database Records for AllMusic
  Creating Database Records for MusicBrainz
  Creating Database Records for LastFM
  Creating Database Records for RockCorner
  Creating Database Records for DatPiff
  Creating Database Records for AceBootlegs
  Creating Databas

/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/90-Metadata.p  	90 121145
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/91-Metadata.p  	91 122464
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/92-Metadata.p  	92 123822
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/93-Metadata.p  	93 125147
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/94-Metadata.p  	94 126441
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/95-Metadata.p  	95 127761
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/96-Metadata.p  	96 129054
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/97-Metadata.p  	97 130404
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/98-Metadata.p  	98 131757
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/99-Metadata.p  	99 133080



Saving 133080 entries to /Users/tgadfort/Music/Discog/db-musicbrainz/ArtistIDToRef.p

Saving data to /Users/tgadfort/Music/Discog/db-musicbrainz/ArtistIDToRef.p
  --> This file is 6.8MB.
Saved data

/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/55-MediaMetadata.p 	69396     0         324971    
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/56-MediaMetadata.p 	70719     0         331620    
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/57-MediaMetadata.p 	72049     0         336933    
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/58-MediaMetadata.p 	73357     0         342882    
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/59-MediaMetadata.p 	74675     0         349265    
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/6-MediaMetadata.p 	76002     0         355627    
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/60-MediaMetadata.p 	77404     0         362247    
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/61-MediaMetadata.p 	78746     0         368482    
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/62-MediaMetadata.p 	80043     0         375042    
/Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/63

/Volumes/Piggy/Discog/artists-lastfm-db/metadata/2-Metadata.p  	12 14632
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/20-Metadata.p  	13 15762
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/21-Metadata.p  	14 16913
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/22-Metadata.p  	15 18037
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/23-Metadata.p  	16 19130
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/24-Metadata.p  	17 20283
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/25-Metadata.p  	18 21386
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/26-Metadata.p  	19 22555
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/27-Metadata.p  	20 23675
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/28-Metadata.p  	21 24781
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/29-Metadata.p  	22 25864
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/3-Metadata.p  	23 26927
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/30-Metadata.p  	24 28011
/Volumes/Piggy/Discog/artists-lastfm-db/

  --> This file is 20.5MB.
Current Time is Thu Sep 10, 2020 23:01:17 for Creating Artist DBs
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/0-MediaMetadata.p 	1105      1987      1987      
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/1-MediaMetadata.p 	2226      3920      3920      
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/10-MediaMetadata.p 	3389      6508      6508      
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/11-MediaMetadata.p 	4503      8799      8799      
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/12-MediaMetadata.p 	5619      10631     10631     
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/13-MediaMetadata.p 	6805      12738     12738     
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/14-MediaMetadata.p 	7926      15021     15021     
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/15-MediaMetadata.p 	9014      17051     17051     
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/16-MediaMetadata.p 	10180     19240     19240     
/Volumes/

/Volumes/Piggy/Discog/artists-lastfm-db/metadata/85-MediaMetadata.p 	93313     175703    175703    
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/86-MediaMetadata.p 	94385     177831    177831    
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/87-MediaMetadata.p 	95474     179992    179992    
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/88-MediaMetadata.p 	96537     181925    181925    
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/89-MediaMetadata.p 	97591     183868    183868    
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/9-MediaMetadata.p 	98700     185882    185882    
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/90-MediaMetadata.p 	99793     188162    188162    
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/91-MediaMetadata.p 	100835    190059    190059    
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/92-MediaMetadata.p 	101897    192191    192191    
/Volumes/Piggy/Discog/artists-lastfm-db/metadata/93-MediaMetadata.p 	103044    194332    194332    
/

/Volumes/Piggy/Discog/artists-discogs-db/metadata/62-Metadata.p  	59 450398
/Volumes/Piggy/Discog/artists-discogs-db/metadata/63-Metadata.p  	60 457989
/Volumes/Piggy/Discog/artists-discogs-db/metadata/64-Metadata.p  	61 465637
/Volumes/Piggy/Discog/artists-discogs-db/metadata/65-Metadata.p  	62 473207
/Volumes/Piggy/Discog/artists-discogs-db/metadata/66-Metadata.p  	63 480737
/Volumes/Piggy/Discog/artists-discogs-db/metadata/67-Metadata.p  	64 488349
/Volumes/Piggy/Discog/artists-discogs-db/metadata/68-Metadata.p  	65 495719
/Volumes/Piggy/Discog/artists-discogs-db/metadata/69-Metadata.p  	66 503228
/Volumes/Piggy/Discog/artists-discogs-db/metadata/7-Metadata.p  	67 510774
/Volumes/Piggy/Discog/artists-discogs-db/metadata/70-Metadata.p  	68 518208
/Volumes/Piggy/Discog/artists-discogs-db/metadata/71-Metadata.p  	69 525657
/Volumes/Piggy/Discog/artists-discogs-db/metadata/72-Metadata.p  	70 533136
/Volumes/Piggy/Discog/artists-discogs-db/metadata/73-Metadata.p  	71 540624
/Volumes/Pigg

/Volumes/Piggy/Discog/artists-discogs-db/metadata/37-MediaMetadata.p 	240621    526396    2262450   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/38-MediaMetadata.p 	248004    541629    2333051   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/39-MediaMetadata.p 	255415    558916    2404630   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/4-MediaMetadata.p 	262983    575526    2478688   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/40-MediaMetadata.p 	270404    592820    2547336   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/41-MediaMetadata.p 	278034    610225    2622626   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/42-MediaMetadata.p 	285525    627912    2692529   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/43-MediaMetadata.p 	293026    645032    2762270   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/44-MediaMetadata.p 	300372    661160    2830561   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/45-MediaMetadata.p 	307681    677023    28

Current Time is Thu Sep 10, 2020 23:10:31 for 
=================================== Creating Artist Album DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 751972 Artists
	Shape --> (751972, 1)
DataFrame Shape is (751972, 1)
Saving Master Artist Albums DB File: /Users/tgadfort/Music/Discog/db-discogs/MasterSlimArtistAlbumsDB.p
  --> This file is 373.2MB.
Current Time is Thu Sep 10, 2020 23:12:23 for Done with 
=================================== Creating Artist Album DB ===================================
Process [Done with 
=================================== Creating Artist Album DB ===================================] took 1.9 minutes.


Known DB



In [ ]:
maindb = mainDB(mdb=mdb, create=False, debug=True)

In [ ]:
maindb.setDBFull()

# Metadata

In [ ]:
%load_ext autoreload
%autoreload

from multiprocessing import Pool
from mainDB import mainDB
import time
mainDB = mainDB()
dbdata = mainDB.dbdata

def parseArtistsAM(modVal, force=False, doExtra=False):
    dbdata["AllMusic"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsAM.parseArtistModValFiles(modVal, force=force)

def parseArtistsDC(modVal, force=True, doExtra=False):
    dbdata["Discogs"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsDC.parseArtistModValFiles(modVal, force=force)

def parseArtistsMB(modVal, force=False, doExtra=False):
    dbdata["MusicBrainz"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsMB.parseArtistModValFiles(modVal, force=force)

def parseArtistsAB(modVal, force=False, doExtra=False):
    artsAB.parseArtistFiles(force=force)
    
def parseArtistsDP(modVal, force=False, doExtra=False):
    dbdata['DatPiff']['Artists'].parseArtistFiles()
    #artsDP.parseArtistFiles(force=force)

def parseArtistsRM(modVal, force=False, doExtra=False):
    dbdata["RateYourMusic"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsRM.parseArtistModValFiles(modVal, force=force)

def parseArtistsLM(modVal, force=False, doExtra=False):
    dbdata["LastFM"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsLM.parseArtistModValFiles(modVal, force=force)

def parseArtistsRC(modVal, force=False, doExtra=False):
    dbdata["RockCorner"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsRC.parseArtistModValFiles(modVal, force=force)

def parseArtistsCL(modVal, force=False, doExtra=False):
    dbdata["CDandLP"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsCL.parseArtistModValFiles(modVal, force=force)

def parseArtistsMS(modVal, force=False, doExtra=False):
    artsMS.parseArtistFiles(force=force)

def parseArtistsMT(modVal, force=False, doExtra=False):
    artsMT.parseArtistModValFiles(modVal, force=force)

    
def parseArtistsParallel(db, nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    if db == "Discogs":
        result = pool.map_async(parseArtistsDC, range(100))
    elif db == "AllMusic":
        result = pool.map_async(parseArtistsAM, range(100))
    elif db == "MusicBrainz":
        result = pool.map_async(parseArtistsMB, range(100))
    elif db == "AceBootlegs":
        result = pool.map_async(parseArtistsAB, [None])
    elif db == "DatPiff":
        result = pool.map_async(parseArtistsDP, [None])
    elif db == "RateYourMusic":
        dbdata["RateYourMusic"]["Artists"].parseDownloadedFiles()
        result = pool.map_async(parseArtistsRM, range(100))
    elif db == "LastFM":
        result = pool.map_async(parseArtistsLM, range(100))
    elif db == "RockCorner":
        result = pool.map_async(parseArtistsRC, range(100))
    elif db == "CDandLP":
        result = pool.map_async(parseArtistsCL, range(100))
        #result = pool.map_async(parseArtistsCL, range(56,72))
        #result = pool.map_async(parseArtistsCL, [55,25,26])
    elif db == "MusicStack":
        result = pool.map_async(parseArtistsMS, [None])
    elif db == "MetalStorm":
        result = pool.map_async(parseArtistsMT, range(100))
    else:
        raise ValueError("[{0}] is not recognized as a DB".format(db))

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

In [ ]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel(db="AllMusic", nProcs=3)
#parseArtistsParallel(db="MusicBrainz", nProcs=3)
parseArtistsParallel(db="Discogs", nProcs=4)
#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=3)
#parseArtistsParallel(db="LastFM", nProcs=3)
#parseArtistsParallel(db="RockCorner", nProcs=3)
#parseArtistsParallel(db="CDandLP", nProcs=3)
#parseArtistsParallel(db="MusicStack", nProcs=1)
elapsed(start2, cmt2)
elapsed(start, cmt)

# Download Stuff

In [ ]:
mdb = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=False, debug=True)
dbdata = maindb.dbdata

In [ ]:
ifile = "unmatchedArtists4.p"
noMatches = getFile(ifile)
print(len(noMatches))
dbs = ['Discogs', 'AllMusic', 'MusicBrainz', 'LastFM']
dbs = ['LastFM']

#noMatches=['Go-A', 'PM', 'Qango']

from time import sleep
maxI = -1
for i,artist in enumerate(noMatches):
    print("="*100)
    print("="*100)
    print("="*100)
    print("="*100)
    print(i,'/',len(noMatches),'\t',artist)
    if i <= maxI:
        continue
    for db in dbs:
        print("="*100)
        print(i,'/',len(noMatches),'\t',artist,'===>',db)
        try:
            dbdata[db]["Artists"].searchForArtist(artist)
        except:
            sleep(5)